# Final Project 
## Complete Machine Learning Pipeline for MIMIC Classification

### Mariajose Argote


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [4]:
# Generate a random seed for reproducibility
# np.random.seed() has a range of [0, 2**32 - 1] for the seed value
print(np.random.randint(0, 2**30))

SEED = 244459055

np.random.seed(SEED)
#540921260


540921260


## Import MIMIC Data

In [9]:
# Import data

train = pd.read_csv('MIMIC III dataset HEF/mimic_train_HEF.csv')
test= pd.read_csv('MIMIC III dataset HEF/mimic_test_HEF.csv')


# Display first few rows of the dataset

print(train.info())

train.describe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20885 entries, 0 to 20884
Data columns (total 44 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   HOSPITAL_EXPIRE_FLAG  20885 non-null  int64  
 1   subject_id            20885 non-null  int64  
 2   hadm_id               20885 non-null  int64  
 3   icustay_id            20885 non-null  int64  
 4   HeartRate_Min         18698 non-null  float64
 5   HeartRate_Max         18698 non-null  float64
 6   HeartRate_Mean        18698 non-null  float64
 7   SysBP_Min             18677 non-null  float64
 8   SysBP_Max             18677 non-null  float64
 9   SysBP_Mean            18677 non-null  float64
 10  DiasBP_Min            18676 non-null  float64
 11  DiasBP_Max            18676 non-null  float64
 12  DiasBP_Mean           18676 non-null  float64
 13  MeanBP_Min            18699 non-null  float64
 14  MeanBP_Max            18699 non-null  float64
 15  MeanBP_Mean        

,HOSPITAL_EXPIRE_FLAG,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,...,TempC_Max,TempC_Mean,SpO2_Min,SpO2_Max,SpO2_Mean,Glucose_Min,Glucose_Max,Glucose_Mean,Diff,LOS
count,20885.000000,20885.000000,20885.000000,20885.000000,18698.000000,18698.000000,18698.000000,18677.000000,18677.000000,18677.000000,...,18388.000000,18388.000000,18682.000000,18682.000000,18682.000000,20632.000000,20632.000000,20632.000000,20885.000000,20885.000000
mean,0.112282,58950.496098,150082.402298,250202.495523,69.705904,105.239801,85.180250,91.110564,150.725920,119.145423,...,37.428781,36.751717,91.007494,99.555883,96.866685,106.781975,182.129604,138.856428,-51617.069830,3.701046
std,0.315720,25299.439535,28898.479845,28909.806302,14.869840,20.922613,15.318208,17.532534,23.833793,16.701503,...,0.799897,0.603476,7.399390,1.159792,2.333108,35.178811,92.665603,44.933145,10686.395846,5.175721
min,0.000000,23.000000,100001.000000,200001.000000,2.000000,39.000000,34.714286,5.000000,46.000000,46.000000,...,30.800000,30.666667,1.000000,57.000000,47.666667,2.000000,42.000000,42.000000,-72740.274440,0.056600
25%,0.000000,41132.000000,125157.000000,225153.000000,60.000000,90.000000,74.272727,81.000000,134.000000,107.100000,...,36.944444,36.388889,90.000000,100.000000,95.812500,86.000000,126.000000,110.750000,-60864.454110,1.165400
50%,0.000000,60441.000000,150152.000000,250452.000000,69.000000,103.000000,84.131183,90.000000,148.000000,116.901961,...,37.333333,36.731481,92.000000,100.000000,97.142857,102.000000,161.000000,128.666667,-51561.703460,2.020800
75%,0.000000,80286.000000,175017.000000,275303.000000,79.000000,118.000000,95.185588,101.000000,164.000000,129.465116,...,37.888889,37.092593,94.000000,100.000000,98.346154,121.000000,206.000000,154.578571,-42327.560030,3.915800
max,1.000000,99999.000000,199999.000000,299998.000000,141.000000,280.000000,163.875000,181.000000,323.000000,202.172414,...,42.000000,40.238333,100.000000,100.000000,100.000000,563.000000,2440.000000,771.900000,-32157.494580,101.739000


In [16]:
print(test.info())
test.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5221 entries, 0 to 5220
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   subject_id      5221 non-null   int64  
 1   hadm_id         5221 non-null   int64  
 2   icustay_id      5221 non-null   int64  
 3   HeartRate_Min   4676 non-null   float64
 4   HeartRate_Max   4676 non-null   float64
 5   HeartRate_Mean  4676 non-null   float64
 6   SysBP_Min       4670 non-null   float64
 7   SysBP_Max       4670 non-null   float64
 8   SysBP_Mean      4670 non-null   float64
 9   DiasBP_Min      4669 non-null   float64
 10  DiasBP_Max      4669 non-null   float64
 11  DiasBP_Mean     4669 non-null   float64
 12  MeanBP_Min      4674 non-null   float64
 13  MeanBP_Max      4674 non-null   float64
 14  MeanBP_Mean     4674 non-null   float64
 15  RespRate_Min    4675 non-null   float64
 16  RespRate_Max    4675 non-null   float64
 17  RespRate_Mean   4675 non-null   f

,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,DiasBP_Min,...,TempC_Min,TempC_Max,TempC_Mean,SpO2_Min,SpO2_Max,SpO2_Mean,Glucose_Min,Glucose_Max,Glucose_Mean,Diff
count,5221.000000,5221.000000,5221.000000,4676.000000,4676.000000,4676.000000,4670.000000,4670.000000,4670.000000,4669.000000,...,4583.000000,4583.000000,4583.000000,4670.000000,4670.000000,4670.000000,5163.000000,5163.000000,5163.000000,5221.000000
mean,58870.629573,150248.186746,249925.293239,69.942686,105.279940,85.221128,90.862392,150.547966,118.763143,44.142643,...,36.041879,37.441470,36.755418,90.828480,99.553319,96.857538,107.823494,376.578346,163.521697,-51606.334303
std,25320.956194,28679.538155,28763.030681,15.013524,21.247956,15.484126,17.529140,23.575450,16.490767,11.695481,...,0.762967,0.797153,0.595941,7.660437,1.516541,2.406957,34.312518,13915.510710,1739.829727,10666.881712
min,107.000000,100033.000000,200011.000000,1.000000,42.000000,36.000000,10.000000,62.000000,51.750000,6.000000,...,22.777778,32.555556,32.222222,1.000000,42.000000,42.000000,15.000000,32.000000,32.000000,-72639.620300
25%,41055.000000,125246.000000,225118.000000,60.000000,90.000000,74.035131,81.000000,134.000000,106.752976,37.000000,...,35.666667,36.944444,36.409722,90.000000,100.000000,95.840227,87.000000,126.000000,111.500000,-60840.730090
50%,60057.000000,150492.000000,249759.000000,69.000000,104.000000,83.957428,90.000000,148.000000,116.807407,44.000000,...,36.111111,37.333333,36.738095,92.000000,100.000000,97.134848,102.000000,160.000000,129.000000,-51612.812060
75%,80313.000000,174983.000000,274576.000000,79.000000,118.000000,95.477143,101.000000,164.000000,128.862844,51.000000,...,36.500000,37.888889,37.091705,94.000000,100.000000,98.333333,122.000000,205.000000,155.060976,-42552.065720
max,99992.000000,199967.000000,299979.000000,143.000000,220.000000,155.571429,162.000000,290.000000,195.250000,97.000000,...,39.388889,41.111111,39.527778,100.000000,100.000000,100.000000,374.000000,999999.000000,125110.125000,-32176.628480


In [22]:
# Check train/test column consistency
target = "HOSPITAL_EXPIRE_FLAG"
leaky_columns = ['LOS', 'DOD', 'DISCHTIME', 'DEATHTIME'] 

# Columns to use as features in train
feature_columns = [c for c in train.columns if c != target and c not in leaky_columns]

# 1) Check that test has all feature columns
missing_in_test = set(feature_columns) - set(test.columns)
extra_in_test   = set(test.columns) - set(feature_columns)
print("Missing in test:", missing_in_test)
print("Extra in test  :", extra_in_test)

# 2) Reorder test columns to match train's feature order
test = test[feature_columns]


#checking type consistency
shared = set(train.columns) & set(test.columns)
dtype_diff = {
    col: (train[col].dtype, test[col].dtype)
    for col in shared
    if train[col].dtype != test[col].dtype
}
print("Columns with different dtypes:", dtype_diff)

Missing in test: set()
Extra in test  : set()
Columns with different dtypes: {}
